<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/fluidflow/FIVcalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
%%capture
!pip install neqsim

**Flow-Induced Vibrations (FIV)**

Flow-induced vibrations are dynamic responses of structures when subjected to fluid flow. These vibrations can be caused by various mechanisms, including:

*   **Vortex Shedding:** As fluid flows past a bluff body (an object with a blunt shape), vortices are shed alternately from each side. This shedding creates oscillating forces perpendicular to the flow direction, which can induce vibrations in the body.
*   **Buffeting:** This occurs when turbulent flow imparts random pressure fluctuations on a structure, leading to random vibrations.
*   **Galloping:** This is a self-excited vibration that can occur in structures with non-circular cross-sections, like power lines or bridge decks. It's driven by aerodynamic forces that increase with the amplitude of vibration.
*   **Flutter:** Aeroelastic instability that can occur in flexible structures like aircraft wings. It's a coupling between aerodynamic forces and the structure's elastic response.


# FlowInducedVibrationAnalyser

The `FlowInducedVibrationAnalyser` class is a measurement device within the Neqsim framework designed to assess the likelihood of flow-induced vibration (FIV) in a pipeline. High-velocity turbulent fluids can generate significant kinetic energy, leading to low-frequency vibrations that may damage piping and supports. This class implements methods to quantify the risk of such vibrations.

## Constructors

### FlowInducedVibrationAnalyser(PipeBeggsAndBrills pipe)

Creates a `FlowInducedVibrationAnalyser` instance associated with a specific `PipeBeggsAndBrills` object.

*   **Parameters:**
    *   `pipe`: The pipeline segment to be analyzed.

### FlowInducedVibrationAnalyser(String name, PipeBeggsAndBrills pipe)

Creates a `FlowInducedVibrationAnalyser` instance with a specified name, associated with a `PipeBeggsAndBrills` object.

*   **Parameters:**
    *   `name`: A custom name for the analyzer.
    *   `pipe`: The pipeline segment to be analyzed.

## Methods

### displayResult()

This method is intended to display the analysis results. *Note: This method is currently not implemented.*

### getMeasuredValue(String unit)

Calculates and returns a value representing the likelihood of flow-induced vibration. The calculation method is determined by the `method` property.

*   **Parameters:**
    *   `unit`: The desired unit for the returned value (currently not used).
*   **Returns:**
    *   A `double` representing the calculated Likelihood of Failure (LOF) or Flow-induced Root Mean Square (FRMS) value.

#### Likelihood of Failure (LOF) Method

This method, based on Energy Institute guidelines, calculates a Likelihood of Failure (LOF) score, which is a screening value to assess the risk of vibration-induced fatigue. The LOF is determined by comparing the kinetic energy of the flow to the pipe's resistance to vibration.

The LOF is calculated as follows:

$LOF = \frac{\rho_{mix} \cdot v_{mix}^2 \cdot FVF}{F_v}$

Where:

*   $\rho_{mix}$: Density of the fluid mixture.
*   $v_{mix}$: Velocity of the fluid mixture.
*   $FVF$: A factor accounting for the influence of the gas volume fraction (GVF) on the vibration potential. It is calculated as:
    *   If $GVF \le 0.88$:
        $FVF = 1.0$
    *   If $0.88 < GVF \le 0.99$:
        $FVF = -27.882 \cdot GVF^2 + 45.545 \cdot GVF - 17.495$
    *   If $GVF > 0.99$:
        $FVF = \sqrt{\frac{\mu_{mix}}{\sqrt{0.001}}}$
        Where $\mu_{mix}$ is the mixture viscosity.

*   $F_v$: The flow-induced vibration factor, which represents the pipe's structural resistance to vibration. It is a function of the pipe's geometry and support arrangement. The calculation is:
    $F_v = \alpha \cdot \left(\frac{D_{ext}}{t}\right)^\beta$
    Where:
    *   $D_{ext}$: External diameter of the pipe.
    *   $t$: Pipe wall thickness.
    *   $\alpha$ and $\beta$: Coefficients determined by the `supportArrangement` property. The values are empirically derived and depend on the stiffness of the pipe supports.

| Support Arrangement | $\alpha$ | $\beta$ |
| :--- | :--- | :--- |
| Stiff | $446187 + 646 \cdot D_{ext} + 9.17 \cdot 10^{-4} \cdot D_{ext}^3$ | $0.1 \cdot \ln(D_{ext}) - 1.3739$ |
| Medium stiff | $283921 + 370 \cdot D_{ext}$ | $0.1106 \cdot \ln(D_{ext}) - 1.501$ |
| Medium | $150412 + 209 \cdot D_{ext}$ | $0.0815 \cdot \ln(D_{ext}) - 1.3269$ |
| Flexible | $41.21 \cdot \ln(D_{ext}) + 49397$ | $0.0815 \cdot \ln(D_{ext}) - 1.3842$ |

#### Flow-induced Root Mean Square (FRMS) Method

This method provides an alternative assessment of vibration.

*   If $GVF < 0.8$:
    The method returns the Gas Volume Fraction (GVF).

For cases where the Gas Volume Fraction (GVF) is higher than or equal to 0.8, a common empirical formula to assess the risk of flow-induced vibration is:

* $FRMS = (1 + 5 \cdot (1 - GVF)) \cdot D^{1.6} \cdot C_{FRMS} \cdot \rho_{liq}^{0.6} \cdot v_{mix}^{1.2}$

Where:
*   **$GVF$**: Gas Volume Fraction (the ratio of the gas volumetric flow rate to the total volumetric flow rate).
*   **$D$**: Internal pipe diameter.
*   **$C_{FRMS}$**: A user-defined constant, often referred to as the FRMS Constant.
*   **$\rho_{liq}$**: Density of the liquid phase.
*   **$v_{mix}$**: Velocity of the multiphase mixture.


### getMethod()

Returns the current calculation method ("LOF" or "FRMS").

*   **Returns:**
    *   A `String` indicating the active method.

### setMethod(String method)

Sets the calculation method to be used.

*   **Parameters:**
    *   `method`: The desired method ("LOF" or "FRMS").

### setSegment(int segment)

Specifies the pipeline segment to be analyzed.

*   **Parameters:**
    *   `segment`: The index of the pipe segment.

### setFRMSConstant(double frms)

Sets the constant used in the FRMS calculation.

*   **Parameters:**
    *   `frms`: The value for the `FRMSConstant`.

### setSupportArrangement(String arrangement)

Defines the type of pipe support arrangement, which influences the $F_v$ calculation.

*   **Parameters:**
    *   `arrangement`: A `String` representing the support stiffness ("Stiff", "Medium stiff", "Medium", or "Flexible").

### setSupportDistance(Double distance)

Sets the distance between pipe supports. *Note: This property is not currently used in the calculations.*

*   **Parameters:**
    *   `distance`: The support distance.

# NeqSim examples

In [33]:
from neqsim import jneqsim

In [34]:
from neqsim.thermo import fluid, TPflash
from neqsim.process import stream

# Create a simple thermodynamic system with methane/ethane
thermoSystem = fluid("srk")
thermoSystem.addComponent("methane", 0.90)
thermoSystem.addComponent("ethane", 0.10)
thermoSystem.addComponent("nC10", 0.10)
thermoSystem.addComponent("water", 1.10)
thermoSystem.setMixingRule("classic")
thermoSystem.setMultiPhaseCheck(True)
thermoSystem.setTemperature(298.15, "K")
thermoSystem.setPressure(70.0, "bara")
thermoSystem.setTotalFlowRate(50000.0, "kg/hr")

# Perform a flash calculation to determine phase properties
TPflash(thermoSystem)
thermoSystem.initPhysicalProperties()

# Create a stream from the thermodynamic system
stream1 = stream('stream 1', thermoSystem)
stream1.setName("test stream")
stream1.run()

# Create a pipe using the Beggs and Brill correlation
pipe = jneqsim.process.equipment.pipeline.PipeBeggsAndBrills('pipe1', stream1)
pipe.setDiameter(0.1)
pipe.setLength(50.0)
pipe.setElevation(0.0)
pipe.setPipeWallRoughness(1.0e-5)
pipe.setThickness(0.01)
pipe.setNumberOfIncrements(10)
pipe.run()

print("Fluid properties and pipe parameters are set up.")
print(f"Density: {thermoSystem.getDensity('kg/m3')} kg/m³")
print(f"Viscosity: {thermoSystem.getViscosity('Pas')} Pa.s")
print(f"Pipe Diameter: {pipe.getDiameter()} m")
print(f"Pipe Length: {pipe.getLength()} m")

analyzerDefaultSegment = jneqsim.process.measurementdevice.FlowInducedVibrationAnalyser("Default segment analyzer", pipe)
analyzerDefaultSegment.setMethod("LOF")
analyzerDefaultSegment.setSupportArrangement("Stiff");
fiv = analyzerDefaultSegment.getMeasuredValue()
print("Stiff segment LOF value: " + str(fiv))
analyzerDefaultSegment.setMethod("FRMS")
frms = analyzerDefaultSegment.getMeasuredValue()
print("Stiff segment FRMS value: " + str(frms))

analyzerDefaultSegment = jneqsim.process.measurementdevice.FlowInducedVibrationAnalyser("Default segment analyzer", pipe)
analyzerDefaultSegment.setMethod("LOF")
analyzerDefaultSegment.setSupportArrangement("Medium");
fiv = analyzerDefaultSegment.getMeasuredValue()
print("Medium segment LOF value: " + str(fiv))
analyzerDefaultSegment.setMethod("FRMS")
frms = analyzerDefaultSegment.getMeasuredValue()
print("Medium segment FRMS value: " + str(frms))

Fluid properties and pipe parameters are set up.
Density: 178.40877112695262 kg/m³
Viscosity: 0.0004862460287701426 Pa.s
Pipe Diameter: 0.1 m
Pipe Length: 50.0 m
Stiff segment LOF value: 0.3691737646474194
Stiff segment FRMS value: 116.11908495913589
Medium segment LOF value: 1.2252761881358707
Medium segment FRMS value: 116.11908495913589


# Use of E300 fluid and MFM measurement for calculation of FRMS/FIV

In [35]:
!wget https://raw.githubusercontent.com/EvenSol/NeqSim-Colab/master/notebooks/PVT/example2.txt

--2025-08-10 21:11:12--  https://raw.githubusercontent.com/EvenSol/NeqSim-Colab/master/notebooks/PVT/example2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8411 (8.2K) [text/plain]
Saving to: ‘example2.txt.2’

example2.txt.2      100%[===================>]   8.21K  --.-KB/s    in 0s      

2025-08-10 21:11:12 (87.6 MB/s) - ‘example2.txt.2’ saved [8411/8411]



In [36]:
file1 = open('example2.txt', "r")
FileContent = file1.read()
print(FileContent)

﻿-- Dummy EOS for FluidMagic Test case
-- 
-- Copyright (C) 2022 Equinor 


-- Units
METRIC
-- Number of components:
NCOMPS
22 /
-- Equation of state
EOS
PR /
PRCORR


-- Reservoir temperature (C)
RTEMP
     90.00 /


-- Standard Conditions (C and bara)
STCOND
   15.00000    1.01325  /


-- Component names
CNAMES
N2
CO2
C1
C2
C3
iC4
C4
iC5
C5
C6
C7
C8
C9
C10-C12
C13-C14
C15-C17
C18-C21
C22-C28
C29-C36
C37-C45
C46-C58
C59-C80 /
-- Tc (K)
TCRIT
   126.200
   304.200
   190.600
   305.400
   369.800
   408.100
   425.200
   460.400
   469.600
   507.400
   548.083
   568.470
   592.686
   631.845
   680.299
   727.035
   774.284
   851.846
   943.373
  1038.592
  1152.236
  1317.304 /
-- Pc (Bar)
PCRIT
   33.9439
   73.7646
   46.0015
   48.8387
   42.4552
   36.4770
   37.9969
   33.8426
   33.7412
   29.6882
   29.4519
   27.6423
   25.5535
   22.7296
   20.0143
   18.1224
   16.7108
   15.1759
   14.0297
   13.2891
   12.7370
   12.2645 /
-- Omega
ACF
   0.04000
   0.22500
   0.00800
 

In [37]:
from neqsim.thermo.thermoTools import readEclipseFluid, printFrame,TPflash
fluid1 = readEclipseFluid('example2.txt')
fluid1.addComponent('water', 1.0)
fluid1.setMixingRule('classic')
fluid1.setMultiPhaseCheck(True)

reservoirPressure = 100
fluid1.setPressure(reservoirPressure, 'bara')
reservoirTemperature = 90.0 # C
fluid1.setTemperature(reservoirTemperature, 'C')

TPflash(fluid1)
printFrame(fluid1)

| 0                    | 1          | 2           | 3           | 4           | 5   | 6               |
|:---------------------|:-----------|:------------|:------------|:------------|:----|:----------------|
|                      | total      | GAS         | OIL         | AQUEOUS     |     |                 |
| nitrogen             | 1.956E-3   | 9.64402E-3  | 1.36208E-3  | 9.29945E-8  |     | [mole fraction] |
| CO2                  | 1.505E-3   | 4.58157E-3  | 2.26469E-3  | 5.29129E-6  |     | [mole fraction] |
| methane              | 2.01637E-1 | 7.80036E-1  | 2.32744E-1  | 2.41212E-6  |     | [mole fraction] |
| ethane               | 3.81705E-2 | 9.07607E-2  | 6.85926E-2  | 7.5885E-9   |     | [mole fraction] |
| propane              | 3.9876E-2  | 6.01381E-2  | 8.66085E-2  | 2.75051E-10 |     | [mole fraction] |
| i-butane             | 5.969E-3   | 6.38668E-3  | 1.40919E-2  | 3.39241E-13 |     | [mole fraction] |
| n-butane             | 2.04645E-2 | 1.83615E-2  | 4.98377E-2  

In [38]:
from neqsim import jneqsim


mpmPressure = 150.0 #bara
mpmTemperature = 90.0 #C

feed_stream = jneqsim.process.equipment.stream.Stream('stream 1', fluid1)
feed_stream.setPressure(mpmPressure, 'bara')
feed_stream.setTemperature(mpmTemperature, 'C')
feed_stream.run()

gas_rate_mfm = 100000.0 #kg/hr
oil_rate_mfm = 100000.0 #kg/hr
water_rate_mfm = 10000.0 #kg/hr

flowadjuster1 = jneqsim.process.equipment.util.FlowRateAdjuster('flow adjuster 1', feed_stream)
flowadjuster1.setAdjustedFlowRates(gas_rate_mfm, oil_rate_mfm, water_rate_mfm, 'kg/hr')
flowadjuster1.run()

valve_out_pressure = 100.0

valve1 = jneqsim.process.equipment.valve.ThrottlingValve('valve 1', flowadjuster1.getOutletStream())
valve1.setPressure(valve_out_pressure, 'bara')
valve1.run()

print('valve outlet temperature ', valve1.getOutletStream().getTemperature('C'), 'C')

print()

printFrame(valve1.getOutletStream().getFluid())




valve outlet temperature  83.89992583297402 C

| 0                    | 1          | 2                 | 3                 | 4                 | 5   | 6               |
|:---------------------|:-----------|:------------------|:------------------|:------------------|:----|:----------------|
|                      | total      | GAS               | OIL               | AQUEOUS           |     |                 |
| nitrogen             | 8.80519E-3 | 1.1247E-2         | 1.60489E-3        | 8.32866E-8        |     | [mole fraction] |
| CO2                  | 3.6316E-3  | 4.3394E-3         | 2.22854E-3        | 4.47648E-6        |     | [mole fraction] |
| methane              | 6.39168E-1 | 7.92469E-1        | 2.42093E-1        | 1.7935E-6         |     | [mole fraction] |
| ethane               | 7.43331E-2 | 8.47465E-2        | 6.70382E-2        | 4.54331E-9        |     | [mole fraction] |
| propane              | 5.54139E-2 | 5.62792E-2        | 8.6146E-2         | 1.47476E-10       |  

In [39]:
# Create a pipe using the Beggs and Brill correlation
pipe = jneqsim.process.equipment.pipeline.PipeBeggsAndBrills('pipe1', valve1.getOutletStream())
pipe.setDiameter(0.2) #m
pipe.setLength(50.0) #m
pipe.setElevation(0.0) #m
pipe.setPipeWallRoughness(10.0e-6)
pipe.setThickness(0.02) #m
pipe.setNumberOfIncrements(1)
pipe.run()

fluid = valve1.getOutletStream().getFluid()
print("Fluid properties and pipe parameters are set up.")
print(f"Density: {fluid.getDensity('kg/m3')} kg/m³")
print(f"Viscosity: {fluid.getViscosity('Pas')} Pas")
print(f"Pipe Diameter: {pipe.getDiameter()} m")
print(f"Pipe Length: {pipe.getLength()} m")

analyzerDefaultSegment = jneqsim.process.measurementdevice.FlowInducedVibrationAnalyser("Default segment analyzer", pipe)
analyzerDefaultSegment.setMethod("LOF")
analyzerDefaultSegment.setSupportArrangement("Stiff");
lof = analyzerDefaultSegment.getMeasuredValue()
print("Stiff segment LOF value: " + str(lof))
analyzerDefaultSegment.setMethod("FRMS")
frms = analyzerDefaultSegment.getMeasuredValue()
print("Stiff segment FRMS value: " + str(frms))

Fluid properties and pipe parameters are set up.
Density: 161.13811135082977 kg/m³
Viscosity: 0.0001072628828243631 Pas
Pipe Diameter: 0.2 m
Pipe Length: 50.0 m
Stiff segment LOF value: 0.2674566148720452
Stiff segment FRMS value: 294.41074069812475


Example of direct calculation in python using neqsim properties

In [40]:
import numpy as np

def get_frms_neqsim(stream, A):
    fluid = stream.getFluid()
    fluid.initPhysicalProperties('density')

    if(fluid.hasPhaseType('oil')):
      m_o = fluid.getPhase('oil').getFlowRate('kg/hr')
      rho_o = fluid.getPhase('oil').getDensity('kg/m3')
      q_o = m_o / rho_o
    else:
      rho_o = 1.0
      m_o = 0.0
      q_o = 0.0

    if(fluid.hasPhaseType('aqueous')):
      m_w = fluid.getPhase('aqueous').getFlowRate('kg/hr')
      rho_w = fluid.getPhase('aqueous').getDensity('kg/m3')
      q_w = m_w / rho_w
    else:
      rho_w = 1.0
      m_w = 0.0
      q_w = 0.0

    wlr = q_w / (q_o + q_w)
    rho_l = rho_w * wlr + rho_o * (1 - wlr)

    if(fluid.hasPhaseType('gas')):
      m_g = fluid.getPhase('gas').getFlowRate('kg/hr')
      rho_g = fluid.getPhase('gas').getDensity('kg/m3')
      q_g = m_g / rho_g
      vel_gas = q_g / (A * 3600)
    else:
      rho_g = 1.0
      q_g = 0.0
      vel_gas = 0.0
      m_g = 0.0

    m_tot = stream.getFlowRate('kg/hr')
    rho_tot = fluid.getDensity('kg/m3')
    q_tot = m_tot / rho_tot
    vel_mix = q_tot / (A * 3600)
    gvf = q_g / q_tot
    D = np.sqrt(4 * A / np.pi)
    Frms_const = 6.4

    return min(1, 5 * (1 - gvf)) * (D ** 1.6) * Frms_const * (rho_l ** 0.6) * (vel_mix ** 1.2), gvf

A_pipe = 3.14*pipe.getDiameter()*pipe.getDiameter()/4.0
print('gvf ', get_frms_neqsim(valve1.getOutletStream(), A_pipe)[1])
print('frms ', get_frms_neqsim(valve1.getOutletStream(), A_pipe)[0])

gvf  0.887300290314517
frms  270.96721932956984
